In [1]:
import random
import os
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import ndcg_score, average_precision_score
import glob
from sklearn.neighbors import NearestNeighbors
import pickle
import scipy.sparse as sp
from scipy.sparse import coo_matrix
from implicit.als import AlternatingLeastSquares

In [2]:
# Charger le fichier articles_embeddings.pickle
embeddings_path = "../data/articles_embeddings.pickle"
with open(embeddings_path, 'rb') as f:
    articles_embeddings = pickle.load(f)

# Vérification de la forme de la matrice d'embeddings
print(f"Type des données : {type(articles_embeddings)}")
print(f"Nombre d'articles : {articles_embeddings.shape[0]}")
print(f"Nombre de dimensions par vecteur : {articles_embeddings.shape[1]}")

# Affichage des 5 premiers vecteurs avec leur longueur
print("Aperçu des 5 premiers vecteurs d'embedding :")
for i, embedding in enumerate(articles_embeddings[:5]):
    print(f"Article {i + 1} - Taille du vecteur : {len(embedding)} - Embedding : {embedding[:10]}...")

Type des données : <class 'numpy.ndarray'>
Nombre d'articles : 364047
Nombre de dimensions par vecteur : 250
Aperçu des 5 premiers vecteurs d'embedding :
Article 1 - Taille du vecteur : 250 - Embedding : [-0.16118301 -0.95723313 -0.13794445  0.05085534  0.83005524  0.90136534
 -0.33514765 -0.55956066 -0.50060284  0.16518293]...
Article 2 - Taille du vecteur : 250 - Embedding : [-0.52321565 -0.974058    0.73860806  0.15523443  0.626294    0.48529708
 -0.71565676 -0.8979958  -0.35974663  0.3982462 ]...
Article 3 - Taille du vecteur : 250 - Embedding : [-0.61961854 -0.9729604  -0.20736018 -0.12886102  0.04474759 -0.387535
 -0.73047674 -0.06612612 -0.75489885 -0.24200428]...
Article 4 - Taille du vecteur : 250 - Embedding : [-0.7408434  -0.97574896  0.39169782  0.6417378  -0.26864457  0.19174537
 -0.82559335 -0.71059096 -0.04009941 -0.11051404]...
Article 5 - Taille du vecteur : 250 - Embedding : [-0.2790515  -0.97231525  0.68537366  0.11305604  0.23831514  0.27191275
 -0.56881577  0.34119

## Qu'est-ce que cela signifie concrètement ?
- Chaque ligne représente un article.
- Les 250 valeurs décrivent ses caractéristiques textuelles sous forme numérique.
- Plus deux vecteurs sont proches (en termes de distance), plus les articles qu'ils représentent sont similaires.

In [3]:
# Chemin du dossier contenant les fichiers de clics
clicks_path = "../data/clicks/"

# Lister tous les fichiers "clicks_hour_***.csv"
clicks_files = sorted([f for f in os.listdir(clicks_path) if f.startswith("clicks_hour_")])

# Charger tous les fichiers et les fusionner en un seul DataFrame
clicks_list = []
for file in clicks_files:
    file_path = os.path.join(clicks_path, file)
    df = pd.read_csv(file_path)
    clicks_list.append(df)

# Fusionner toutes les données
clicks_df = pd.concat(clicks_list, ignore_index=True)

# Afficher des informations sur les données combinées
print(f"Nombre total de lignes : {len(clicks_df)}")
print(f"Aperçu des premières lignes :")
print(clicks_df.head())
print("Informations générales :")
print(clicks_df.info())

Nombre total de lignes : 2988181
Aperçu des premières lignes :
  user_id        session_id  session_start session_size click_article_id  \
0       0  1506825423271737  1506825423000            2           157541   
1       0  1506825423271737  1506825423000            2            68866   
2       1  1506825426267738  1506825426000            2           235840   
3       1  1506825426267738  1506825426000            2            96663   
4       2  1506825435299739  1506825435000            2           119592   

  click_timestamp click_environment click_deviceGroup click_os click_country  \
0   1506826828020                 4                 3       20             1   
1   1506826858020                 4                 3       20             1   
2   1506827017951                 4                 1       17             1   
3   1506827047951                 4                 1       17             1   
4   1506827090575                 4                 1       17             1   


In [4]:
# Convertir les colonnes importantes en type numérique
clicks_df['user_id'] = pd.to_numeric(clicks_df['user_id'], errors='coerce')
clicks_df['click_article_id'] = pd.to_numeric(clicks_df['click_article_id'], errors='coerce')
clicks_df['click_timestamp'] = pd.to_datetime(clicks_df['click_timestamp'], unit='ms', errors='coerce')

# Supprimer les lignes avec des valeurs manquantes
clicks_df.dropna(subset=['user_id', 'click_article_id', 'click_timestamp'], inplace=True)

# Supprimer les doublons (un clic par utilisateur et article)
clicks_df.drop_duplicates(subset=['user_id', 'click_article_id'], inplace=True)

# Vérification finale
print(f"Nombre total de lignes après nettoyage : {len(clicks_df)}")
print(f"Aperçu des données nettoyées :")
print(clicks_df.head())

Nombre total de lignes après nettoyage : 2950710
Aperçu des données nettoyées :
   user_id        session_id  session_start session_size  click_article_id  \
0        0  1506825423271737  1506825423000            2            157541   
1        0  1506825423271737  1506825423000            2             68866   
2        1  1506825426267738  1506825426000            2            235840   
3        1  1506825426267738  1506825426000            2             96663   
4        2  1506825435299739  1506825435000            2            119592   

          click_timestamp click_environment click_deviceGroup click_os  \
0 2017-10-01 03:00:28.020                 4                 3       20   
1 2017-10-01 03:00:58.020                 4                 3       20   
2 2017-10-01 03:03:37.951                 4                 1       17   
3 2017-10-01 03:04:07.951                 4                 1       17   
4 2017-10-01 03:04:50.575                 4                 1       17   

  clic

In [5]:
# Créer un dictionnaire pour associer chaque article à son embedding
article_ids = np.arange(articles_embeddings.shape[0])  # Les IDs des articles sont dans l'ordre
article_embeddings_dict = dict(zip(article_ids, articles_embeddings))

# Associer chaque clic d'utilisateur à l'embedding de l'article
def get_embedding(article_id):
    return article_embeddings_dict.get(article_id, np.zeros(250))

clicks_df['article_embedding'] = clicks_df['click_article_id'].apply(get_embedding)

# Créer un profil pour chaque utilisateur en moyennant les embeddings des articles cliqués
user_profiles = clicks_df.groupby('user_id')['article_embedding'].apply(
    lambda x: np.mean(np.stack(x), axis=0)
)

# Afficher un aperçu des profils utilisateurs
print(f"Nombre de profils utilisateurs créés : {len(user_profiles)}")
print("Aperçu des 5 premiers profils :")
for user_id, profile in user_profiles.head().items():
    print(f"Utilisateur {user_id} - Vecteur (taille {len(profile)}): {profile[:10]}...")

Nombre de profils utilisateurs créés : 322897
Aperçu des 5 premiers profils :
Utilisateur 0 - Vecteur (taille 250): [-0.2103363  -0.96357316 -0.19693483 -0.03603265 -0.35537797 -0.06501008
 -0.26539654  0.2746106   0.05667167 -0.07157854]...
Utilisateur 1 - Vecteur (taille 250): [-0.30157247 -0.9656159  -0.37045392 -0.30028948 -0.2534356  -0.2310031
 -0.25263694  0.01866199  0.1814787  -0.04513927]...
Utilisateur 2 - Vecteur (taille 250): [-0.6273024  -0.9667226  -0.1068523  -0.55297124  0.31660053 -0.20927826
 -0.5314152  -0.16470814 -0.11809298  0.28117475]...
Utilisateur 3 - Vecteur (taille 250): [-0.5952114  -0.9658984  -0.25120276 -0.3851927  -0.1014897  -0.3768533
 -0.50448596 -0.09360608  0.01020017 -0.27796048]...
Utilisateur 4 - Vecteur (taille 250): [ 0.01588762 -0.9651615  -0.10582644 -0.28574044 -0.08624995 -0.58963054
 -0.3324241   0.0146182  -0.1247003  -0.5037674 ]...


In [6]:
# Filtrer les profils utilisateurs valides
valid_user_profiles = {user_id: profile for user_id, profile in user_profiles.items() if profile.shape == (250,)}

# Créer une matrice de profils, en remplaçant les vecteurs manquants par des zéros
user_profiles_matrix = np.array([valid_user_profiles.get(user_id, np.zeros(250)) for user_id in user_profiles.index])

# Vérification de la forme
print(f"Nombre de profils valides : {len(valid_user_profiles)}")
print(f"Forme de la matrice des profils : {user_profiles_matrix.shape}")

Nombre de profils valides : 322897
Forme de la matrice des profils : (322897, 250)


In [7]:
# Nombre de profils par lot pour éviter la surcharge
batch_size = 1000

# Stocker les recommandations
recommendations = []

# Calculer la similarité par lot
for i in range(0, user_profiles_matrix.shape[0], batch_size):
    batch = user_profiles_matrix[i:i + batch_size]
    similarity_batch = cosine_similarity(batch, articles_embeddings)

    # Trouver les 5 articles les plus proches pour chaque utilisateur du lot
    top_5_batch = np.argsort(-similarity_batch, axis=1)[:, :5]

    # Stocker les résultats
    for user_idx, articles in enumerate(top_5_batch):
        user_id = user_profiles.index[i + user_idx]
        recommendations.append((user_id, list(articles)))

# Créer un DataFrame des recommandations
recommendations_df = pd.DataFrame(recommendations, columns=['user_id', 'recommended_articles'])

# Afficher un aperçu
print(f"Nombre total de recommandations générées : {len(recommendations_df)}")
print("Aperçu des 5 premières recommandations :")
print(recommendations_df.head())

Nombre total de recommandations générées : 322897
Aperçu des 5 premières recommandations :
   user_id                      recommended_articles
0        0  [162235, 160966, 162230, 155943, 160079]
1        1  [237201, 346278, 156501, 234334, 280341]
2        2       [31400, 31920, 25856, 31927, 29828]
3        3  [229025, 234127, 235395, 236309, 233769]
4        4  [192998, 188797, 193907, 193242, 188911]


In [ ]:
# Sauvegarde des recommandations dans un fichier CSV
output_path = "../data/user_recommendations.csv"
recommendations_df.to_csv(output_path, index=False)

print(f"Les recommandations ont été sauvegardées ici : {output_path}")

💾 Les recommandations ont été sauvegardées ici : ../data/user_recommendations.csv


In [ ]:
# Chemin du fichier sauvegardé
input_path = "../data/user_recommendations.csv"

# Charger les recommandations depuis le fichier CSV
recommendations_df = pd.read_csv(input_path)

# Afficher un aperçu pour vérifier le chargement
print(f"Fichier chargé avec succès depuis : {input_path}")
print(f"Nombre total de recommandations chargées : {len(recommendations_df)}")
print("\nAperçu des 5 premières lignes :")
print(recommendations_df.head())

📂 Fichier chargé avec succès depuis : ../data/user_recommendations.csv
✨ Nombre total de recommandations chargées : 322897

📊 Aperçu des 5 premières lignes :
   user_id                               recommended_articles
0        0  [np.int64(162235), np.int64(160966), np.int64(...
1        1  [np.int64(237201), np.int64(346278), np.int64(...
2        2  [np.int64(31400), np.int64(31920), np.int64(25...
3        3  [np.int64(229025), np.int64(234127), np.int64(...
4        4  [np.int64(192998), np.int64(188797), np.int64(...


In [ ]:
# Convertir les articles en entiers simples avec NumPy
recommendations_df['recommended_articles'] = recommendations_df['recommended_articles'].apply(
    lambda x: [int(np.int64(article)) for article in eval(x)]
)

# Vérification
print(f"Après nettoyage avec NumPy, voici un aperçu des 5 premières lignes :")
print(recommendations_df.head())

✨ Après nettoyage avec NumPy, voici un aperçu des 5 premières lignes :
   user_id                      recommended_articles
0        0  [162235, 160966, 162230, 155943, 160079]
1        1  [237201, 346278, 156501, 234334, 280341]
2        2       [31400, 31920, 25856, 31927, 29828]
3        3  [229025, 234127, 235395, 236309, 233769]
4        4  [192998, 188797, 193907, 193242, 188911]


# Modèle
Maintenant qu'on a fait des essais en calcul direct, on va créer un modèle.

In [11]:
# Charger les données des articles
embeddings_path = "../data/articles_embeddings.pickle"
with open(embeddings_path, 'rb') as f:
    articles_embeddings = pickle.load(f)

In [12]:
# Entraîner un modèle KNN pour le Content-Based Filtering
print("Entraînement du modèle KNN...")
knn_model = NearestNeighbors(n_neighbors=5, metric='cosine')
knn_model.fit(articles_embeddings)

Entraînement du modèle KNN...


NearestNeighbors(metric='cosine')

In [13]:
# Sauvegarder le modèle entraîné
model_path = "../data/cbf_model.pkl"
with open(model_path, 'wb') as f:
    pickle.dump(knn_model, f)

print(f"Modèle CBF sauvegardé ici : {model_path}")

Modèle CBF sauvegardé ici : ../data/cbf_model.pkl


In [14]:
# Chemin du modèle sauvegardé
model_path = "../data/cbf_model.pkl"

# Charger le modèle KNN
with open(model_path, 'rb') as f:
    knn_model = pickle.load(f)

In [15]:
# Exemple : Recommander des articles similaires à un article connu (ici, article ID = 100)
article_id = 100  # Change cet ID pour tester différents articles
article_embedding = articles_embeddings[article_id].reshape(1, -1)

# Trouver les 5 articles les plus similaires
distances, indices = knn_model.kneighbors(article_embedding)

# Afficher les recommandations
print(f"Articles recommandés pour l'article {article_id} :")
for i, index in enumerate(indices[0]):
    print(f"{i + 1}. Article ID : {index} | Distance : {distances[0][i]:.4f}")

Articles recommandés pour l'article 100 :
1. Article ID : 100 | Distance : 0.0000
2. Article ID : 1085 | Distance : 0.1295
3. Article ID : 935 | Distance : 0.1330
4. Article ID : 343 | Distance : 0.1359
5. Article ID : 1270 | Distance : 0.1373


In [16]:
# Fonction de prédiction locale
def get_recommendations(article_id, n_recommendations=5):
    try:
        # Récupérer l'embedding de l'article demandé
        article_embedding = articles_embeddings[article_id].reshape(1, -1)

        # Trouver les articles les plus proches
        distances, indices = knn_model.kneighbors(article_embedding, n_neighbors=n_recommendations)

        # Afficher les recommandations
        recommendations = [{"article_id": int(idx), "distance": float(dist)} for idx, dist in zip(indices[0], distances[0])]

        return recommendations

    except IndexError:
        return f"L'article ID {article_id} n'existe pas dans les embeddings."

In [17]:
# Test de la fonction
article_id_test = 100
print(f"Recommandations pour l'article {article_id_test} :")
print(get_recommendations(article_id_test))

Recommandations pour l'article 100 :
[{'article_id': 100, 'distance': 0.0}, {'article_id': 1085, 'distance': 0.12954258918762207}, {'article_id': 935, 'distance': 0.13304436206817627}, {'article_id': 343, 'distance': 0.13586843013763428}, {'article_id': 1270, 'distance': 0.13728368282318115}]


# CF

In [18]:
# Chemin du dossier contenant les fichiers de clics
clicks_path = "../data/clicks/"

# Lister tous les fichiers de clics
clicks_files = sorted([f for f in os.listdir(clicks_path) if f.startswith("clicks_hour_")])

# Charger tous les fichiers et les fusionner
clicks_list = []
for file in clicks_files:
    file_path = os.path.join(clicks_path, file)
    df = pd.read_csv(file_path)
    clicks_list.append(df[['user_id', 'click_article_id']])

# Fusionner les DataFrames en un seul
clicks_df = pd.concat(clicks_list, ignore_index=True)

In [19]:
# Charger les données des clics
clicks_df['interaction'] = 1

# Convertir les user_id et article_id en indices numériques pour la matrice sparse
user_mapping = {user_id: idx for idx, user_id in enumerate(clicks_df['user_id'].unique())}
article_mapping = {article_id: idx for idx, article_id in enumerate(clicks_df['click_article_id'].unique())}

clicks_df['user_idx'] = clicks_df['user_id'].map(user_mapping)
clicks_df['article_idx'] = clicks_df['click_article_id'].map(article_mapping)

# Créer une matrice sparse directement
sparse_user_item_matrix = coo_matrix(
    (clicks_df['interaction'], (clicks_df['user_idx'], clicks_df['article_idx'])),
    shape=(len(user_mapping), len(article_mapping))
).tocsr()

# Afficher les dimensions et la mémoire utilisée
print(f"Matrice sparse créée avec succès !")
print(f"Dimensions : {sparse_user_item_matrix.shape}")
print(f"Nombre de cellules non nulles (clics) : {sparse_user_item_matrix.nnz}")

Matrice sparse créée avec succès !
Dimensions : (322897, 46033)
Nombre de cellules non nulles (clics) : 2950710


In [20]:
# Convertir la matrice en format compatible avec ALS (float32)
sparse_user_item_matrix = sparse_user_item_matrix.astype('float32')

# Entraîner le modèle ALS
print("Entraînement du modèle ALS...")
als_model = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=10)

# Adapter le modèle aux données
als_model.fit(sparse_user_item_matrix.T)

print("Modèle ALS entraîné avec succès !")

Entraînement du modèle ALS...


e:\Openclassroom\AI_Engineer\Projet_10\work_folder\env_projet_10\lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 24 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
e:\Openclassroom\AI_Engineer\Projet_10\work_folder\env_projet_10\lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.02101588249206543 seconds
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

Modèle ALS entraîné avec succès !


In [21]:
# Recommander des articles pour un utilisateur (exemple avec user_id = 0)
user_id = 0

if user_id in user_mapping:
    user_idx = user_mapping[user_id]

    # Obtenir les recommandations
    recommended = als_model.recommend(user_idx, sparse_user_item_matrix[user_idx], N=5)

    print(f"Articles recommandés pour l'utilisateur {user_id} :")
    for rec in recommended:
        article_id, score = int(rec[0]), rec[1]  # Conversion en entier

        # Convertir l'index en ID original
        article_original_id = list(article_mapping.keys())[list(article_mapping.values()).index(article_id)]

        print(f" - Article ID : {article_original_id}, Score : {score:.4f}")
else:
    print(f"L'utilisateur {user_id} n'existe pas dans la matrice.")

Articles recommandés pour l'utilisateur 0 :
 - Article ID : 202736, Score : 15867.0000
 - Article ID : 157541, Score : 0.0757


In [22]:
# Chemin pour sauvegarder le modèle
model_path = "../data/cf_model.pkl"

# Sauvegarde du modèle ALS
with open(model_path, 'wb') as f:
    pickle.dump(als_model, f)

print(f"Modèle ALS sauvegardé avec succès ici : {model_path}")

Modèle ALS sauvegardé avec succès ici : ../data/cf_model.pkl


# Combinaison des modèles

In [23]:
# Chemins des modèles sauvegardés
cbf_model_path = "../data/cbf_model.pkl"
cf_model_path = "../data/cf_model.pkl"

# Charger le modèle CBF (KNN)
with open(cbf_model_path, 'rb') as f:
    knn_model = pickle.load(f)
print(f"Modèle CBF chargé depuis : {cbf_model_path}")

# Charger le modèle CF (ALS)
with open(cf_model_path, 'rb') as f:
    als_model = pickle.load(f)
print(f"Modèle CF chargé depuis : {cf_model_path}")

Modèle CBF chargé depuis : ../data/cbf_model.pkl
Modèle CF chargé depuis : ../data/cf_model.pkl


In [24]:
# Poids pour la combinaison
alpha = 0.5  # 50% CBF, 50% CF

# Exemple avec un utilisateur spécifique
user_id = 0

# Recommandations CF (ALS)
def get_cf_recommendations(user_id, n=5):
    if user_id not in user_mapping:
        return {}

    user_idx = user_mapping[user_id]
    cf_recommendations = als_model.recommend(user_idx, sparse_user_item_matrix[user_idx], N=n)

    # Extraire les ID et scores, en s'assurant qu'il n'y ait pas d'autres valeurs
    cf_scores = {}
    for rec in cf_recommendations:
        article_id = int(rec[0])  # Premier élément : ID de l'article
        score = float(rec[1])     # Deuxième élément : Score
        cf_scores[article_id] = score

    return cf_scores

In [25]:
# Recommandations CBF (KNN)
def get_cbf_recommendations(user_id, n=5):
    # Utiliser le profil utilisateur pour recommander (basé sur les embeddings)
    if user_id not in user_profiles:
        return {}

    user_profile = user_profiles[user_id].reshape(1, -1)
    distances, indices = knn_model.kneighbors(user_profile, n_neighbors=n)

    # Convertir les indices en IDs d'articles
    cbf_recommendations = {int(idx): 1 - dist for idx, dist in zip(indices[0], distances[0])}
    return cbf_recommendations

In [26]:
# Combinaison des scores
def get_hybrid_recommendations(user_id, n=5):
    cf_scores = get_cf_recommendations(user_id, n * 2)  # Plus large pour mieux fusionner
    cbf_scores = get_cbf_recommendations(user_id, n * 2)

    # Fusionner les scores
    all_articles = set(cf_scores.keys()).union(set(cbf_scores.keys()))
    hybrid_scores = {}

    for article_id in all_articles:
        cf_score = cf_scores.get(article_id, 0)
        cbf_score = cbf_scores.get(article_id, 0)
        hybrid_score = alpha * cbf_score + (1 - alpha) * cf_score
        hybrid_scores[article_id] = hybrid_score

    # Trier par score décroissant
    sorted_recommendations = sorted(hybrid_scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_recommendations[:n]

In [27]:
# Test des recommandations hybrides
hybrid_recommendations = get_hybrid_recommendations(user_id)

print(f"Recommandations hybrides pour l'utilisateur {user_id} :")
for article_id, score in hybrid_recommendations:
    print(f" - Article ID : {article_id}, Score Hybride : {score:.4f}")

Recommandations hybrides pour l'utilisateur 0 :
 - Article ID : 9261, Score Hybride : 7933.5000
 - Article ID : 162235, Score Hybride : 0.4374
 - Article ID : 160966, Score Hybride : 0.4245
 - Article ID : 162230, Score Hybride : 0.4209
 - Article ID : 155943, Score Hybride : 0.4209


In [ ]:
# Créer un dictionnaire avec les modèles et les paramètres
hybrid_model = {
    'cbf_model': knn_model, # Modèle KNN pour le Content-Based Filtering
    'cf_model': als_model, # Modèle ALS utilisé pour le Collaborative Filtering
    'alpha': alpha, # Poids pour la combinaison => 50% CBF, 50% CF
    'user_mapping': user_mapping, # Dictionnaire qui associe chaque user_id à un indice numérique
    'article_mapping': article_mapping # Pareil, mais pour les articles
}

# Chemin pour sauvegarder le modèle hybride
hybrid_model_path = "../data/hybrid_model.pkl"

# Sauvegarde
with open(hybrid_model_path, 'wb') as f:
    pickle.dump(hybrid_model, f)

print(f"Modèle hybride sauvegardé avec succès ici : {hybrid_model_path}")

Modèle hybride sauvegardé avec succès ici : ../data/hybrid_model.pkl
